In [2]:
import os
import glob
from dotenv import load_dotenv
import nibabel as nib
from nibabel.orientations import aff2axcodes
import h5py
import pandas as pd
import numpy as np
import scipy.io as sio
from tqdm import tqdm

from utils.registration import nifti_info, correct4registration, slice2chunk_align
from utils.preproc import FUSCleaner

from utils.glm import run_glm_analysis
from nilearn.glm.contrasts import compute_contrast
from scipy.ndimage import affine_transform
import SimpleITK as sitk

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from joblib import Parallel, delayed

In [3]:
load_dotenv()
base_dir = os.getenv('BASE_DIR')
task_dir = os.path.join(base_dir, 'task_data')
data_dir = os.path.join(task_dir, 'video_live_social')
videosocial2d_dir = os.path.join(data_dir, '20241128_videosocial_2d')
videosocial3d_dir = os.path.join(data_dir, '20241201_videosocial_3d')
livesocial2d_dir = os.path.join(data_dir, '20241130_livesocial_2d')
livesocial3d_dir = os.path.join(data_dir, '20241202_livesocial_3d')

In [12]:
dir = videosocial2d_dir
brain_data = nib.load(glob.glob(os.path.join(dir, '*.nii'))[0])
print(brain_data.shape)
event_file = glob.glob(os.path.join(dir, '*.mat'))[0]
event_mat = sio.loadmat(event_file)['mldata'][0][0][0]
print(event_mat.shape)

pixdim[1,2,3] should be positive; setting to abs of pixdim values


(128, 1, 143, 9000)
(204,)


In [4]:
infile = os.path.join(livesocial3d_dir, 'zion_angio_3d.nii.gz')
outfile = os.path.join(livesocial3d_dir, 'zion_angio_3d_corrected.nii.gz')
correct4registration(infile, outfile)
nifti_info, nifti_info(outfile)

Corrected Affine:
 [[  0.           0.2         -0.          -0.        ]
 [  0.11         0.          -0.          -0.        ]
 [  0.           0.           0.09856    -13.99551964]
 [  0.           0.           0.           1.        ]]
('A', 'R', 'S')
[[  0.           0.2          0.           0.        ]
 [  0.11         0.           0.           0.        ]
 [  0.           0.          -0.09856    -13.99551964]
 [  0.           0.           0.           1.        ]]


(<function utils.registration.nifti_info(file)>,
 ((128, 26, 143),
  array([[  0.        ,   0.2       ,   0.        ,   0.        ],
         [  0.11      ,   0.        ,   0.        ,   0.        ],
         [  0.        ,   0.        ,  -0.09856   , -13.99551964],
         [  0.        ,   0.        ,   0.        ,   1.        ]]),
  ('A', 'R', 'I')))

In [17]:
file = os.path.join(livesocial3d_dir, 'zion_angio_3d.nii')
img = nib.load(file)
data = img.get_fdata()

data.shape,img.affine, aff2axcodes(img.affine)

pixdim[1,2,3] should be positive; setting to abs of pixdim values


((128, 26, 143),
 array([[-0.11      ,  0.        ,  0.        ,  6.98499996],
        [ 0.        ,  0.2       ,  0.        , -2.50000004],
        [ 0.        ,  0.        ,  0.09856   , -6.99775986],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 ('L', 'A', 'S'))

In [18]:
file = os.path.join(livesocial3d_dir, 'zion_angio_3d.nii.gz')
img = nib.load(file)
data = img.get_fdata()

data.shape,img.affine, aff2axcodes(img.affine)

((128, 26, 143),
 array([[  0.11      ,   0.        ,  -0.        ,  -0.        ],
        [  0.        ,   0.2       ,  -0.        ,  -0.        ],
        [  0.        ,   0.        ,   0.09856   , -13.99551964],
        [  0.        ,   0.        ,   0.        ,   1.        ]]),
 ('R', 'A', 'S'))

In [19]:
file = os.path.join(livesocial3d_dir, 'zion_angio_3d.nii.gz')
img = nib.load(file)
data = img.get_fdata()

print(data.shape)

affine = img.affine

# Create a transformation matrix to swap x and y axes in the affine matrix
# This involves swapping the first and second columns of the affine matrix
transform_matrix = np.array([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, -1, 0],
                            [0, 0, 0, 1]])

transformed_affine = affine @ transform_matrix
print(transformed_affine) 

flipped_data = np.flip(data, axis=2)

new_img = nib.Nifti1Image(flipped_data, transformed_affine, img.header)

output_file = os.path.join(livesocial3d_dir, 'zion_angio_3d_corrected.nii.gz')
nib.save(new_img, output_file)

(128, 26, 143)
[[  0.11         0.           0.           0.        ]
 [  0.           0.2          0.           0.        ]
 [  0.           0.          -0.09856    -13.99551964]
 [  0.           0.           0.           1.        ]]
